In [3]:
from rake_nltk import Rake   # ensure this is installed

import nltk
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
coffee_df = pd.read_csv("Resources/coffee_clean.csv")
coffee_df

,slug,aroma,acid_or_milk,body,flavor,type_with_milk,clean_text,roast_dark,roast_light,roast_medium,...,region_asia_pacific,region_south_america,type_espresso,type_organic,type_fair_trade,type_decaffeinated,type_pod_capsule,type_blend,type_estate,type_with_milk.1
0,ethiopia-deri-kochoha-2,9.0,8.0,9.0,9.0,0,bright crisp sweetli tart citru medley cacao n...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,espresso-14,8.0,9.0,8.0,8.0,1,evalu espresso deepli rich sweetli roast round...,0,0,1,...,1,0,1,0,0,0,0,0,1,1
2,kenya-ruthaka-peaberry,9.0,8.0,9.0,10.0,0,deepli sweet richli savori dark chocol pistach...,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,ethiopia-gora-kone-sidamo,9.0,8.0,9.0,9.0,0,fruit forward richli chocolati raspberri couli...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,specialty-coffee-blend-espresso,9.0,9.0,8.0,9.0,1,evalu espresso rich chocolati sweetli tart dar...,0,0,0,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4882,beanery-blend,8.0,7.0,7.0,7.0,0,light medium roast blend power aroma intens cl...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4883,house-blend,7.0,8.0,6.0,8.0,0,rest tast profil play peek boo around domin ca...,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4884,presidents-private-blend,6.0,5.0,6.0,5.0,0,given medium roast carbon note bit surpris for...,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4885,traditional-roast,7.0,6.0,6.0,6.0,0,rel light roast particularli acidi fact past c...,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
coffee_id_df = pd.read_csv("Resources/coffee_id.csv")
coffee_id_df

,slug,name,roaster,rating,review_date
0,ethiopia-deri-kochoha-2,Ethiopia Deri Kochoha,Flight Coffee Co.,93,2019-01-01
1,espresso-14,Espresso,Doi Chaang Coffee,91,2019-01-01
2,kenya-ruthaka-peaberry,Kenya Ruthaka Peaberry,Temple Coffee and Tea,95,2019-01-01
3,ethiopia-gora-kone-sidamo,Ethiopia Gora Kone Sidamo,Temple Coffee and Tea,93,2019-01-01
4,specialty-coffee-blend-espresso,Specialty Coffee Blend Espresso,Choosy Gourmet,93,2019-01-01
...,...,...,...,...,...
4882,beanery-blend,Beanery Blend,The Coffee Beanery,83,1997-02-01
4883,house-blend,House Blend,Starbucks Coffee,81,1997-02-01
4884,presidents-private-blend,President’s Private Blend,Peerless Coffee,75,1997-02-01
4885,traditional-roast,Traditional Roast,Gevalia,74,1997-02-01


In [38]:
combined_df = coffee_df.merge(coffee_id_df, on='slug')
combined_df

,slug,aroma,acid_or_milk,body,flavor,type_with_milk,clean_text,roast_dark,roast_light,roast_medium,...,type_fair_trade,type_decaffeinated,type_pod_capsule,type_blend,type_estate,type_with_milk.1,name,roaster,rating,review_date
0,ethiopia-deri-kochoha-2,9.0,8.0,9.0,9.0,0,bright crisp sweetli tart citru medley cacao n...,0,0,0,...,0,0,0,0,0,0,Ethiopia Deri Kochoha,Flight Coffee Co.,93,2019-01-01
1,espresso-14,8.0,9.0,8.0,8.0,1,evalu espresso deepli rich sweetli roast round...,0,0,1,...,0,0,0,0,1,1,Espresso,Doi Chaang Coffee,91,2019-01-01
2,kenya-ruthaka-peaberry,9.0,8.0,9.0,10.0,0,deepli sweet richli savori dark chocol pistach...,0,0,1,...,0,0,0,0,0,0,Kenya Ruthaka Peaberry,Temple Coffee and Tea,95,2019-01-01
3,ethiopia-gora-kone-sidamo,9.0,8.0,9.0,9.0,0,fruit forward richli chocolati raspberri couli...,0,0,0,...,0,0,0,0,0,0,Ethiopia Gora Kone Sidamo,Temple Coffee and Tea,93,2019-01-01
4,specialty-coffee-blend-espresso,9.0,9.0,8.0,9.0,1,evalu espresso rich chocolati sweetli tart dar...,0,0,0,...,0,0,0,0,0,1,Specialty Coffee Blend Espresso,Choosy Gourmet,93,2019-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4882,beanery-blend,8.0,7.0,7.0,7.0,0,light medium roast blend power aroma intens cl...,0,0,0,...,0,0,0,0,0,0,Beanery Blend,The Coffee Beanery,83,1997-02-01
4883,house-blend,7.0,8.0,6.0,8.0,0,rest tast profil play peek boo around domin ca...,1,0,0,...,0,0,0,0,0,0,House Blend,Starbucks Coffee,81,1997-02-01
4884,presidents-private-blend,6.0,5.0,6.0,5.0,0,given medium roast carbon note bit surpris for...,0,0,1,...,0,0,0,0,0,0,President’s Private Blend,Peerless Coffee,75,1997-02-01
4885,traditional-roast,7.0,6.0,6.0,6.0,0,rel light roast particularli acidi fact past c...,0,1,0,...,0,0,0,0,0,0,Traditional Roast,Gevalia,74,1997-02-01


In [7]:
# data overview
print('Rows x Columns : ', combined_df.shape[0], 'x', combined_df.shape[1])
print('\nFeatures: ', combined_df.columns.tolist())
print('\nUnique values:')
print(combined_df.nunique())
for col in combined_df.columns:
    print(col, end=': ')
    print(combined_df[col].unique())

Rows x Columns :  4887 x 32

Features:  ['slug', 'aroma', 'acid_or_milk', 'body', 'flavor', 'type_with_milk', 'clean_text', 'roast_dark', 'roast_light', 'roast_medium', 'roast_medium_dark', 'roast_medium_light', 'roast_very_dark', 'roast_nan', 'region_africa_arabia', 'region_caribbean', 'region_central_america', 'region_hawaii', 'region_asia_pacific', 'region_south_america', 'type_espresso', 'type_organic', 'type_fair_trade', 'type_decaffeinated', 'type_pod_capsule', 'type_blend', 'type_estate', 'type_with_milk.1', 'name', 'roaster', 'rating', 'review_date']

Unique values:
slug                      4887
aroma                       33
acid_or_milk                37
body                        29
flavor                      38
type_with_milk               2
clean_text                4886
roast_dark                   2
roast_light                  2
roast_medium                 2
roast_medium_dark            2
roast_medium_light           2
roast_very_dark              2
roast_nan       

In [8]:
# type of entries, how many missing values/null fields
combined_df.info()
print('\nMissing values:  ', combined_df.isnull().sum().values.sum())
combined_df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4887 entries, 0 to 4886
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   slug                    4887 non-null   object 
 1   aroma                   4887 non-null   float64
 2   acid_or_milk            4887 non-null   float64
 3   body                    4887 non-null   float64
 4   flavor                  4887 non-null   float64
 5   type_with_milk          4887 non-null   int64  
 6   clean_text              4887 non-null   object 
 7   roast_dark              4887 non-null   int64  
 8   roast_light             4887 non-null   int64  
 9   roast_medium            4887 non-null   int64  
 10  roast_medium_dark       4887 non-null   int64  
 11  roast_medium_light      4887 non-null   int64  
 12  roast_very_dark         4887 non-null   int64  
 13  roast_nan               4887 non-null   int64  
 14  region_africa_arabia    4887 non-null   

slug                      0
aroma                     0
acid_or_milk              0
body                      0
flavor                    0
type_with_milk            0
clean_text                0
roast_dark                0
roast_light               0
roast_medium              0
roast_medium_dark         0
roast_medium_light        0
roast_very_dark           0
roast_nan                 0
region_africa_arabia      0
region_caribbean          0
region_central_america    0
region_hawaii             0
region_asia_pacific       0
region_south_america      0
type_espresso             0
type_organic              0
type_fair_trade           0
type_decaffeinated        0
type_pod_capsule          0
type_blend                0
type_estate               0
type_with_milk.1          0
name                      0
roaster                   0
rating                    0
review_date               0
dtype: int64

In [9]:
combined_df.describe().T

,count,mean,std,min,25%,50%,75%,max
aroma,4887.0,8.179108,0.997619,2.0,8.0,8.0,9.0,10.0
acid_or_milk,4887.0,7.804277,0.964405,2.0,7.0,8.0,8.0,10.0
body,4887.0,7.893452,0.887634,4.0,7.5,8.0,8.0,10.0
flavor,4887.0,8.259975,1.090489,1.0,8.0,9.0,9.0,10.0
type_with_milk,4887.0,0.138122,0.345063,0.0,0.0,0.0,0.0,1.0
roast_dark,4887.0,0.044199,0.205558,0.0,0.0,0.0,0.0,1.0
roast_light,4887.0,0.085533,0.279702,0.0,0.0,0.0,0.0,1.0
roast_medium,4887.0,0.277062,0.447593,0.0,0.0,0.0,1.0,1.0
roast_medium_dark,4887.0,0.156333,0.363208,0.0,0.0,0.0,0.0,1.0
roast_medium_light,4887.0,0.339881,0.473717,0.0,0.0,0.0,1.0,1.0


In [10]:
# to generate the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(combined_df['clean_text'])
count_matrix

<4887x4968 sparse matrix of type '<class 'numpy.int64'>'
	with 175251 stored elements in Compressed Sparse Row format>

In [22]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1.        , 0.18232629, 0.27547807, ..., 0.17778266, 0.06206214,
        0.13744062],
       [0.18232629, 1.        , 0.31627837, ..., 0.1680535 , 0.13538259,
        0.10660036],
       [0.27547807, 0.31627837, 1.        , ..., 0.15831026, 0.06028857,
        0.13351294],
       ...,
       [0.17778266, 0.1680535 , 0.15831026, ..., 1.        , 0.21561439,
        0.24253563],
       [0.06206214, 0.13538259, 0.06028857, ..., 0.21561439, 1.        ,
        0.28575029],
       [0.13744062, 0.10660036, 0.13351294, ..., 0.24253563, 0.28575029,
        1.        ]])

In [12]:
# to create a Series for movie titles which can be used as indices (each index is mapped to a movie title)
indices = pd.Series(combined_df['name'])
indices[:5]

0              Ethiopia Deri Kochoha
1                           Espresso
2             Kenya Ruthaka Peaberry
3          Ethiopia Gora Kone Sidamo
4    Specialty Coffee Blend Espresso
Name: name, dtype: object

In [13]:
# this function takes in a movie title as input and returns the top 10 recommended (similar) movies

def recommend(name, cosine_sim = cosine_sim):
    recommended_coffees = []
    idx = indices[indices == name].index[0]   # to get the index of the movie title matching the input movie
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)   # similarity scores in descending order
    top_10_indices = list(score_series.iloc[1:11].index)   # to get the indices of top 10 most similar movies
    # [1:11] to exclude 0 (index 0 is the input movie itself)
    
    for i in top_10_indices:   # to append the titles of top 10 similar movies to the recommended_movies list
        recommended_coffees.append(list(combined_df['name'])[i])
        
    return recommended_coffees

recommend('Espresso')

['“Sweety” Espresso Blend',
 'Thai Single-Origin Natural-Process Espresso',
 'Mini-Me Washed Kaffa Forest',
 'Ethiopia Gede Natural',
 'Jade Espresso',
 'Signature Hazelnut Dark Chocolate',
 'Espresso Blend',
 'House Blend Café',
 'Espresso Blend Spring 15',
 'Ethiopia Natural Phoenix Special “Andromeda” Espresso']

In [32]:
combined_df

,slug,aroma,acid_or_milk,body,flavor,type_with_milk,clean_text,roast_dark,roast_light,roast_medium,...,type_fair_trade,type_decaffeinated,type_pod_capsule,type_blend,type_estate,type_with_milk.1,name,roaster,rating,review_date
0,ethiopia-deri-kochoha-2,9.0,8.0,9.0,9.0,0,bright crisp sweetli tart citru medley cacao n...,0,0,0,...,0,0,0,0,0,0,Ethiopia Deri Kochoha,Flight Coffee Co.,93,2019-01-01
1,espresso-14,8.0,9.0,8.0,8.0,1,evalu espresso deepli rich sweetli roast round...,0,0,1,...,0,0,0,0,1,1,Espresso,Doi Chaang Coffee,91,2019-01-01
2,kenya-ruthaka-peaberry,9.0,8.0,9.0,10.0,0,deepli sweet richli savori dark chocol pistach...,0,0,1,...,0,0,0,0,0,0,Kenya Ruthaka Peaberry,Temple Coffee and Tea,95,2019-01-01
3,ethiopia-gora-kone-sidamo,9.0,8.0,9.0,9.0,0,fruit forward richli chocolati raspberri couli...,0,0,0,...,0,0,0,0,0,0,Ethiopia Gora Kone Sidamo,Temple Coffee and Tea,93,2019-01-01
4,specialty-coffee-blend-espresso,9.0,9.0,8.0,9.0,1,evalu espresso rich chocolati sweetli tart dar...,0,0,0,...,0,0,0,0,0,1,Specialty Coffee Blend Espresso,Choosy Gourmet,93,2019-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4882,beanery-blend,8.0,7.0,7.0,7.0,0,light medium roast blend power aroma intens cl...,0,0,0,...,0,0,0,0,0,0,Beanery Blend,The Coffee Beanery,83,1997-02-01
4883,house-blend,7.0,8.0,6.0,8.0,0,rest tast profil play peek boo around domin ca...,1,0,0,...,0,0,0,0,0,0,House Blend,Starbucks Coffee,81,1997-02-01
4884,presidents-private-blend,6.0,5.0,6.0,5.0,0,given medium roast carbon note bit surpris for...,0,0,1,...,0,0,0,0,0,0,President’s Private Blend,Peerless Coffee,75,1997-02-01
4885,traditional-roast,7.0,6.0,6.0,6.0,0,rel light roast particularli acidi fact past c...,0,1,0,...,0,0,0,0,0,0,Traditional Roast,Gevalia,74,1997-02-01


In [39]:
html_df = combined_df.drop(columns=['slug','type_with_milk','type_espresso','type_fair_trade','type_pod_capsule','type_blend','type_estate','type_with_milk.1'])
html_df.head()

,aroma,acid_or_milk,body,flavor,clean_text,roast_dark,roast_light,roast_medium,roast_medium_dark,roast_medium_light,...,region_central_america,region_hawaii,region_asia_pacific,region_south_america,type_organic,type_decaffeinated,name,roaster,rating,review_date
0,9.0,8.0,9.0,9.0,bright crisp sweetli tart citru medley cacao n...,0,0,0,0,1,...,0,0,0,0,0,0,Ethiopia Deri Kochoha,Flight Coffee Co.,93,2019-01-01
1,8.0,9.0,8.0,8.0,evalu espresso deepli rich sweetli roast round...,0,0,1,0,0,...,0,0,1,0,0,0,Espresso,Doi Chaang Coffee,91,2019-01-01
2,9.0,8.0,9.0,10.0,deepli sweet richli savori dark chocol pistach...,0,0,1,0,0,...,0,0,0,0,0,0,Kenya Ruthaka Peaberry,Temple Coffee and Tea,95,2019-01-01
3,9.0,8.0,9.0,9.0,fruit forward richli chocolati raspberri couli...,0,0,0,0,1,...,0,0,0,0,0,0,Ethiopia Gora Kone Sidamo,Temple Coffee and Tea,93,2019-01-01
4,9.0,9.0,8.0,9.0,evalu espresso rich chocolati sweetli tart dar...,0,0,0,0,1,...,0,0,0,0,0,0,Specialty Coffee Blend Espresso,Choosy Gourmet,93,2019-01-01


In [40]:
html_df = html_df.drop(columns=['roast_dark','roast_light','roast_medium','roast_medium_dark','roast_medium_light','roast_very_dark','roast_nan','region_africa_arabia','region_caribbean','region_central_america','region_hawaii','region_asia_pacific','region_south_america','type_organic','type_decaffeinated'])
html_df.head()

,aroma,acid_or_milk,body,flavor,clean_text,name,roaster,rating,review_date
0,9.0,8.0,9.0,9.0,bright crisp sweetli tart citru medley cacao n...,Ethiopia Deri Kochoha,Flight Coffee Co.,93,2019-01-01
1,8.0,9.0,8.0,8.0,evalu espresso deepli rich sweetli roast round...,Espresso,Doi Chaang Coffee,91,2019-01-01
2,9.0,8.0,9.0,10.0,deepli sweet richli savori dark chocol pistach...,Kenya Ruthaka Peaberry,Temple Coffee and Tea,95,2019-01-01
3,9.0,8.0,9.0,9.0,fruit forward richli chocolati raspberri couli...,Ethiopia Gora Kone Sidamo,Temple Coffee and Tea,93,2019-01-01
4,9.0,9.0,8.0,9.0,evalu espresso rich chocolati sweetli tart dar...,Specialty Coffee Blend Espresso,Choosy Gourmet,93,2019-01-01


In [45]:
html_df = html_df.reindex(columns=['name','roaster','clean_text','rating','aroma','acid_or_milk','body','flavor','review_date'])
html_df.head()

,name,roaster,clean_text,rating,aroma,acid_or_milk,body,flavor,review_date
0,Ethiopia Deri Kochoha,Flight Coffee Co.,bright crisp sweetli tart citru medley cacao n...,93,9.0,8.0,9.0,9.0,2019-01-01
1,Espresso,Doi Chaang Coffee,evalu espresso deepli rich sweetli roast round...,91,8.0,9.0,8.0,8.0,2019-01-01
2,Kenya Ruthaka Peaberry,Temple Coffee and Tea,deepli sweet richli savori dark chocol pistach...,95,9.0,8.0,9.0,10.0,2019-01-01
3,Ethiopia Gora Kone Sidamo,Temple Coffee and Tea,fruit forward richli chocolati raspberri couli...,93,9.0,8.0,9.0,9.0,2019-01-01
4,Specialty Coffee Blend Espresso,Choosy Gourmet,evalu espresso rich chocolati sweetli tart dar...,93,9.0,9.0,8.0,9.0,2019-01-01


In [47]:
html_df.to_csv('Resources/html.csv')